<a href="https://colab.research.google.com/github/pvkothapalli/MET-CS777/blob/master/CS777_Cos_pkothapalli_Assignmnet1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Initilaization section for JDK and Spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://mirror.cc.columbia.edu/pub/software/apache/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-preview2-bin-hadoop2.7.tgz
!pip install -q findspark
!rm -rf spark-3.0.0-preview2-bin-hadoop2.7.tgz

In [0]:
#Set up Java and Spark environmnet with spark context
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview2-bin-hadoop2.7"

 

In [0]:
#setting up Spark Context
import findspark
findspark.init("spark-3.0.0-preview2-bin-hadoop2.7")# SPARK_HOME

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()

In [0]:
#download data file ( small size -  Taxi data for 2013) from amazon s3 bucket  as a data set 
!wget https://s3.amazonaws.com/metcs777/taxi-data-sorted-small.csv.bz2


In [217]:
# Load data for  processing for all the tasks
file = "./taxi-data-sorted-small.csv.bz2"
#file = "s3://metcs777/taxi-data-sorted-small.csv.bz2"
# read input text file to RDD
lines = sc.textFile(file)
# splt  the lines of the file with "," separator 
taxis = lines.map(lambda x: x.split(','))

#map columns variables 
taxis_orig_data = taxis.map(lambda p: (p[0], p[1], p[2], p[3], p[4], p[5], p[6], p[7], p[8], p[9],\
                                  p[10], p[11], p[12], p[13], p[14], p[15], p[16]))

#Provide an action to make spark rdd execute and get the count of rows
taxis_orig_data.count()


Py4JJavaError: ignored

In [184]:

# Check for bad data and Filter if  Medallion and hack_license( driver id) columns or any other coulmns are null
# refer to the Description of data for columns. 0 - Medallion, 1 - taxi id, 2 - pickup datetime, 4- trip in sec, 5 - trip distance, 12- surcharge
# 16 - total amount
taxis_filt_data = taxis_orig_data.filter( lambda p : False 
                                         if p[0].strip() == '' or  \
                                          p[1].strip() == '' or \
                                          p[2].strip() == '' or p[3].strip() == '' or \
                                          p[4].strip() == '' or p[5].strip() == '' or \
                                          p[6].strip() == '' or p[7].strip() == '' or \
                                          p[8].strip() == '' or p[9].strip() == '' or \
                                          p[10].strip() == '' or p[11].strip() == '' or \
                                          p[12].strip() == '' or p[13].strip() == '' or \
                                          p[14].strip() == '' or p[15].strip() == '' or \
                                          p[16].strip() == '' else  \
                                         True
                                         )

# Count of Rows
taxis_filt_data.count()

1999999

In [189]:
#4.1 Task1  Compute Top 10 taxis Drivers using Medallion as key, generate ( medallion, no of drivers ) as pairs
from operator import add
# Filter Duplicate records for Medallion and Driver id as data has multiple trips by the same drivers
driver_data_count = taxis_filt_data.map(lambda p: (p[0], p[1])).distinct()
driver_data_count.count()

26398

In [186]:
from operator import add
# Filter Duplicate records for Medallion and Driver id as data has multiple trips by the same drivers
# Get unique Medallion and Driver ids
driver_data_count = taxis_filt_data.map(lambda p: (p[0], p[1])).distinct()

# Build a map of the Medallion with 1 which points  to a unique driver (1) and reduce by Medallion  using add
fil_driver_data_count = driver_data_count.map(lambda p: (p[0], 1)).reduceByKey(add)

#let us take 5 rows
fil_driver_data_count.take(5)
 


[('9836A6780886D5D11F7BA21BAEFD0E38', 3),
 ('74DA41EDA745ED72CEC41C9C4BF3E696', 5),
 ('9406D2C34715E1DA10AD4D4DDADF4DA5', 5),
 ('D7C280941FCD886701B71BE52D350503', 2),
 ('FCC037120B4287337868BA53FD007782', 2)]

In [187]:
# Get the top 10 of the Medalion , Driver counts using top() function 
fil_driver_data_count.top(10, key = lambda p: p[1])


[('65EFB7D02BAD12D5DE757CB5D350944E', 20),
 ('3C08296D0EB7ABE24FB7328DE9B62813', 20),
 ('7DEB25123AE57111F912C0EBF92F1F63', 19),
 ('3B6AE3CF05F34ADC91DC68D20F2EB913', 19),
 ('799153A138F4E8334A1A95AE25040B83', 19),
 ('F36564AB9C6EA3B6373EB0E1680A447A', 19),
 ('55D311AD2752BC278BEF7386B25B28A9', 19),
 ('CD7B02776E69483397952DC5E1F44DFE', 19),
 ('F2A08960199BCDB7EE19411A8E7A4C5D', 18),
 ('9FB7A7C1D7B960D8B17829145C6A1CF2', 18)]

In [205]:
# Get the top 10 of the Medalion , Driver counts using takeOrdered() function
print(' 4.1 Task Top 10 Medallion Driver pairs: ')
fil_driver_data_count.takeOrdered(10, key = lambda p: -p[1])


 4.1 Task Top 10 Medallion Driver pairs: 


[('65EFB7D02BAD12D5DE757CB5D350944E', 20),
 ('3C08296D0EB7ABE24FB7328DE9B62813', 20),
 ('7DEB25123AE57111F912C0EBF92F1F63', 19),
 ('3B6AE3CF05F34ADC91DC68D20F2EB913', 19),
 ('799153A138F4E8334A1A95AE25040B83', 19),
 ('F36564AB9C6EA3B6373EB0E1680A447A', 19),
 ('55D311AD2752BC278BEF7386B25B28A9', 19),
 ('CD7B02776E69483397952DC5E1F44DFE', 19),
 ('F2A08960199BCDB7EE19411A8E7A4C5D', 18),
 ('9FB7A7C1D7B960D8B17829145C6A1CF2', 18)]

In [16]:
# 4.2 Task 2 
# We would like to figure out who the top 10 best drivers are in terms of their average earned money per
# minute spent carrying a customer. The total amount field is the total money earned on a trip. In the end,
# we are interested in computing a set of (driver, money per minute) pairs.
# average earned money per minute spent carrying a customer = (total_amount/duration of trip in secs) * 60
# p[1], p[16]/p[4] * 60

# Build a map of medallion with DriverId, TripTime and Total Paid Amount as a RDD

driver_data_count = taxis_filt_data.map(lambda p: (p[1], (int(p[4]), float(p[16]))))

#Get the count of Rows with multiple Rides forthe drivers
driver_data_count.count()

1999999

In [47]:
#Take 5 row sof the data
driver_data_count.take(5)

[('E7750A37CAB07D0DFF0AF7E3573AC141', (120, 4.5)),
 ('3FF2709163DE7036FCAA4E5A3324E4BF', (0, 27.5)),
 ('778C92B26AE78A9EBDF96B49C67E4007', (120, 5.0)),
 ('BE317B986700F63C43438482792C8654', (120, 5.0)),
 ('7077F9FD5AD649AEACA4746B2537E3FA', (120, 5.0))]

In [194]:
# Compute the sum of the (duration of trip time in seconds)  and sum of the (total amount by the driver)
driver_data_count1 = taxis_filt_data.map(lambda p: (p[1], (int(p[4]), float(p[16])))).reduceByKey(lambda a,b: (a[0]+ b[0], a[1]+ b[1]))
# Action for RDD to execute sc
print ( 'Total 4.1 row count:', driver_data_count1.count())
driver_data_count1.take(2)

Total 4.1 row count: 20354


[('778C92B26AE78A9EBDF96B49C67E4007', (62220, 1377.7799999999997)),
 ('25BA06A87905667AA1FE5990E33F0E2E', (150840, 2696.18))]

In [197]:
# Filter values with sum of Duration time  and sum of Total _amount as values 0
fil_driver_data_count1 =  driver_data_count1.filter(lambda p: True if (int(p[1][0]) != 0 and  float(p[1][1]) != 0 ) else False)
print ( 'Filtered Total 4.2 row count:', fil_driver_data_count1.count())
fil_driver_data_count1.take(2)


Filtered Total 4.2 row count: 20338


[('778C92B26AE78A9EBDF96B49C67E4007', (62220, 1377.7799999999997)),
 ('25BA06A87905667AA1FE5990E33F0E2E', (150840, 2696.18))]

In [202]:
#calculate the average rate per minute by division of sums for each driver
driver_data_count2 = fil_driver_data_count1.map(lambda x: (x[0],round(float((float(x[1][1])/int(x[1][0]))*60),2 )))
driver_data_count2.take(2)

[('778C92B26AE78A9EBDF96B49C67E4007', 1.33),
 ('25BA06A87905667AA1FE5990E33F0E2E', 1.07)]

In [206]:
# Get the top 10 of the average rate per minute  for the drivers
print(' 4.2 Task Top 10   Driver Average Rate pairs: ')
driver_data_count2.top(10, key = lambda p: round(float(p[1]),2))

 4.2 Task Top 10   Driver Average Rate pairs: 


[('57D463B8F4C3382081F206E6869AA095', 3780.0),
 ('69B6FBD28F84175AB1504F6BFF001A49', 2400.0),
 ('0838C4C7DDFD9391AD66E316B5608B26', 1815.0),
 ('30B2ACBAF002305533FF0D31D34A7C06', 702.0),
 ('4C3B2A31227663A59E1AA7B45157160D', 625.0),
 ('A0AC85170C37E1D076ADE05B0238C58E', 540.0),
 ('08026D69508127F4DE855678ABCE7E0A', 375.0),
 ('D3B2DEC5DB78D91D9AFADA53B3B521B5', 330.0),
 ('6E1D7195E38AA7A36B375C1C60AD8632', 317.31),
 ('7826BDE4CE3E44EE1BB7B926A38A4B2A', 279.86)]

In [0]:
# 4.3. Task 3 (8 Points)
# We are interested to know the time of the day (pickup time of day - p[2]) that has the highest profit ratio. 
# Consider the surcharge amount in dollar for each taxi ride (without tip amount) and the distance in
# miles, and sum up the rides for each hour of the day (24 hours) – consider the pickup time for your calculation.
# Profit Ratio = (Surcharge Amount p[12] in US Dollar) / (Travel Distance p[5] in miles)

In [208]:
# filter data that has "0.0" dollars surcharge and 0.00 as trip distance and date has  invalid data as  zeros 
taxis_filt_data1 = taxis_filt_data.filter(lambda p: True if (p[2] != "0000-00-00 00:00:00" and float(p[12]) != 0.0  and   float(p[5]) != 0.0 ) else False)
print('4.3 count of filtered2 rows:', taxis_filt_data1.count())
# Data has multiple days and hours and Key is defined as "Date + hour [00-24]"
# Map data to get date and hour as unique  Key  surcharge, trip_distance
taxis_filt_data2 = taxis_filt_data1.map(lambda x: (x[2][0:10]+x[2][11:13], (float(x[12]), float(x[5]))) )
#print sample 5 rows of data
taxis_filt_data2.take(2)


4.3 count of filtered2 rows: 943112


[('2013-01-0100', (0.5, 0.44)), ('2013-01-0100', (0.5, 0.71))]

In [209]:
#Filter  data if date format ( key)  is null or bad rows of data
taxis_filt_data21 = taxis_filt_data2.filter(lambda p: True if (p[0] != "0000-00-0000" or p[0] != '' or len(p[0] != 12)) else False)
print('4.3 count of filtered3 rows:' ,taxis_filt_data21.count())

4.3 count of filtered3 rows: 943112


In [213]:
#compute sum of surcharge and sum of trip_distance in miles by time of day by hour ( for all the given dates and hours combination  -  defined as key ) using reduceByKey
taxis_filt_data3 = taxis_filt_data21.reduceByKey(lambda x,y: (round(float(x[0]) + float(y[0]),2),round(float(x[1])  + float(y[1]),2)))
print('4.3 Total number of Time of day ( keys):' , taxis_filt_data3.count())
#Execute RDD with action function
taxis_filt_data3.take(2)

4.3 Total number of Time of day ( keys): 208


[('2013-01-0100', (6486.0, 36219.3)), ('2013-01-0101', (7091.0, 46674.43))]

In [214]:
# compute the Profit ratio as sum(surchargeby hourof day)/sum(trip distance by hour of day)
taxis_filt_data4 = taxis_filt_data3.map(lambda x: (x[0],  round(float(float(x[1][0])/float(x[1][1])),2))) 
print('4.3 Total number of Profit Ratio Date-time:' , taxis_filt_data4.count())
taxis_filt_data4.take(3)

4.3 Total number of Profit Ratio Date-time: 208


[('2013-01-0100', 0.18), ('2013-01-0101', 0.15), ('2013-01-0104', 0.14)]

In [215]:
# Get the time of the day that has the highest profit ratio using max function
print('4.3 Time of the day that has the highest profit ratio:' )
taxis_filt_data4.max( lambda x: x[1])
 

4.3 Time of the day that has the highest profit ratio:


('2013-01-0411', 12.5)

In [216]:
# Get the time of the day that has the highest profit ratio using max function using top function
print('4.3 Time of the day that has the highest profit ratio:' )
taxis_filt_data4.top(1, key = lambda p: p[1])

4.3 Time of the day that has the highest profit ratio:


[('2013-01-0411', 12.5)]